In [1]:
# -*- coding: utf-8 -*-
"""
Análise de Painel - Dissertação (Python vs Stata)
Requisitos (pip): pandas, numpy, matplotlib, pyreadstat, linearmodels, scipy, statsmodels, seaborn (opcional para gráficos)
"""
import bootstrap_deps as deps
deps.ensure_in_notebook(requirements_file="requirements.txt")

import os
import re
import unicodedata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# opcional (para gráficos mais bonitos; remova se preferir apenas matplotlib)
try:
    import seaborn as sns
except Exception:
    sns = None

from pyreadstat import read_sas7bdat
from scipy.stats import mstats
from linearmodels.panel import PanelOLS, RandomEffects, PooledOLS
from scipy.stats import chi2

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 180)
pd.set_option('display.max_rows', 200)

# ------------------------------
# Configuração
# ------------------------------
INPUT_PATH = r"/Users/macvini/Library/CloudStorage/OneDrive-Pessoal/Mestrado/base_final_mestrado.sas7bdat"
RESULTS_DIR = "./resultados_python"
os.makedirs(RESULTS_DIR, exist_ok=True)


# ------------------------------
# Utils
# ------------------------------
def slug(s: str) -> str:
    """Normaliza nomes de colunas (sem acentos/símbolos), em minúsculas e com underscores."""
    if s is None:
        return "col"
    s = ''.join(c for c in unicodedata.normalize('NFKD', str(s)) if not unicodedata.combining(c))
    s = re.sub(r'[^0-9a-zA-Z_]+', '_', s).strip('_').lower()
    return s or "col"


def winsorize_series(s: pd.Series, limits=(0.01, 0.01)) -> pd.Series:
    """Winsoriza série preservando índice e dtype float; ignora NaNs."""
    s2 = s.astype(float).copy()
    mask = s2.notna()
    if mask.any():
        s2.loc[mask] = np.asarray(mstats.winsorize(s2.loc[mask].values, limits=limits), dtype=float)
    return s2.astype(float)


def hausman(fe_res, re_res):
    """Teste de Hausman entre FE e RE (aproximação)."""
    b = fe_res.params
    B = re_res.params.reindex_like(b)
    common = b.dropna().index.intersection(B.dropna().index)
    b = b.loc[common]; B = B.loc[common]
    Vb = fe_res.cov.loc[common, common]
    VB = re_res.cov.loc[common, common]
    diff = (b - B).values.reshape(-1,1)
    V = (Vb - VB).values
    try:
        stat = float(diff.T @ np.linalg.pinv(V) @ diff)
    except Exception:
        return np.nan, len(common), np.nan
    dof_ = len(common)
    pval = 1 - chi2.cdf(stat, dof_)
    return stat, dof_, pval


# ------------------------------
# 1) Importação
# ------------------------------
def load_data(path: str) -> pd.DataFrame:
    # Try loading with utf-8, fallback to latin1 if error occurs
    try:
        df, meta = read_sas7bdat(path, encoding='utf-8')
    except Exception:
        df, meta = read_sas7bdat(path, encoding='latin1')
    return df


# ------------------------------
# 2) Preparação & Engenharia
# ------------------------------
def prepare_engineer(df: pd.DataFrame) -> pd.DataFrame:
    # Excluir linhas com missing/vazio para 'cliente' e criar id_cliente
    if 'cliente' in df.columns:
        df = df.dropna(subset=['cliente'])
        df = df[df['cliente'] != '']
        df['id_cliente'] = df.groupby('cliente').ngroup()
    else:
        raise KeyError("Coluna 'cliente' não encontrada.")

    # Data de nascimento -> idade
    if 'DT_NASCIMENTO' in df.columns:
    # Apenas garanta que a coluna seja do tipo datetime, sem conversões complexas
        df['DT_NASCIMENTO'] = pd.to_datetime(df['DT_NASCIMENTO'])
        hoje = pd.to_datetime('today').normalize()
        df['idade'] = (hoje - df['DT_NASCIMENTO']).dt.days / 365.25
        df['idade_int'] = df['idade'].astype(int)
    else:
        df['idade_int'] = np.nan

    # Investimento exterior consolidado
    colunas_invest_ext = [c for c in [
        'INVEST_EXT_RENDA_VARIAVEL', 'INVEST_NO_EXTERIOR', 'INVEST_EXTERIOR', 'INVEST_EXT_RENDA_FIXA'
    ] if c in df.columns]
    df['investimento_exterior'] = df[colunas_invest_ext].sum(axis=1, skipna=True) if colunas_invest_ext else 0.0

    # Sexo dummy (float)
    if 'SEXO' in df.columns:
        df['sexo_dummy'] = df['SEXO'].map({'M': 1, 'F': 0}).astype(float)
    else:
        df['sexo_dummy'] = np.nan

    # Regiões (nomes com underscore)
    regioes = {
        'norte': ["AC", "AP", "AM", "PA", "RO", "RR", "TO"],
        'nordeste': ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
        'sudeste': ["ES", "MG", "RJ", "SP"],
        'sul': ["PR", "RS", "SC"],
        'centro_oeste': ["DF", "GO", "MT", "MS"],
    }
    if 'UF_CADASTRO' in df.columns:
        for regiao, ufs in regioes.items():
            df[f'regiao_{regiao}'] = df['UF_CADASTRO'].isin(ufs).astype(int)
        reg_cols = ['regiao_norte', 'regiao_nordeste', 'regiao_sudeste', 'regiao_sul', 'regiao_centro_oeste']
        df['regiao_codigo'] = df[reg_cols].dot(pd.Series([1,2,3,4,5], index=reg_cols)).astype(int)
    else:
        for regiao in ['norte','nordeste','sudeste','sul','centro_oeste']:
            df[f'regiao_{regiao}'] = 0
        df['regiao_codigo'] = 0

    # Estilo investidor
    if 'NM_TIP_CTRA' in df.columns:
        df['estilo_investidor'] = (df['NM_TIP_CTRA'] == "ESTILO INVESTIDOR").astype(int)
    else:
        df['estilo_investidor'] = 0

    # Estado civil -> grupos
    if 'EST_CIVIL' in df.columns:
        ec_map = {
            1: 'solteiro',
            2: 'casado', 3: 'casado', 4: 'casado', 8: 'casado', 9: 'casado', 11: 'casado', 12: 'casado',
            6: 'separado', 7: 'separado',
            5: 'viuvo'
        }
        df['estado_civil_grupo'] = df['EST_CIVIL'].map(ec_map).fillna('nao_informado')
        ec_dum = pd.get_dummies(df['estado_civil_grupo'], prefix='ec', dtype=int)
        df = pd.concat([df, ec_dum], axis=1)
    else:
        df['estado_civil_grupo'] = 'nao_informado'

    # Escolaridade -> grupos
    if 'ESCOLAR' in df.columns:
        esc_map = {
            1: 'baixa', 2: 'baixa',
            3: 'media', 4: 'media', 9: 'media',
            5: 'alta', 6: 'alta', 7: 'alta', 8: 'alta',
            0: 'missing'
        }
        df['escolaridade_grupo'] = df['ESCOLAR'].map(esc_map).fillna('nao_informado')
        esc_dum = pd.get_dummies(df['escolaridade_grupo'], prefix='esc', dtype=int)
        df = pd.concat([df, esc_dum], axis=1)
    else:
        df['escolaridade_grupo'] = 'nao_informado'

    # Perfil investidor
    if 'CD_PRFL_API' in df.columns:
        df['prfl_codigo'] = df['CD_PRFL_API'].replace(0, 5)
        df['prfl_codigo'] = df['prfl_codigo'].fillna(5).astype(int)
        perfil_map = {1: 'conservador', 2: 'moderado', 3: 'arrojado', 4: 'agressivo', 5: 'nao_resp'}
        df['perfil_grupo'] = df['prfl_codigo'].map(perfil_map)
    else:
        df['perfil_grupo'] = 'nao_resp'

    # Ocupação (regex)
    ocup_map = {
        'Administracao': r'ADMINISTRADOR|CONTADOR|ANALISTA|CONSULTOR|ECONOMISTA',
        'Servidor_Publico': r'SERVIDOR PUBLICO|DEPUTADO|PREFEITO|SECRETARIO|MAGISTRADO|PROCURADOR',
        'Saude': r'MEDICO|ENFERMEIRO|FISIOTERAPEUTA|ODONTOLOGO|FARMACEUTICO|NUTRICIONISTA|FONOAUDIOLOGO|PSICOLOGO|TERAPEUTA',
        'Educacao': r'PROFESSOR|ESTUDANTE|ESTAGIARIO|BOLSISTA|PEDAGOGO',
        'Autonomo_Comercio': r'COMERCIANTE|AMBULANTE|TAXISTA|VENDEDOR|FEIRANTE|REPRESENTANTE COMERCIAL',
        'Agropecuaria': r'AGRICULTOR|PECUARISTA|PESCADOR|AVICULTOR|RURAL|FLORICULTOR|AGRONOMO|AGROPECUARISTA',
        'Industrial': r'MECANICO|ELETRICISTA|OPERADOR|CONSTRUCAO|MARCENEIRO|INDUSTRIARIO|SERRALHEIRO|TECNIC',
        'Justica': r'ADVOGADO|DELEGADO|DEFENSOR|PROMOTOR|JUIZ|OFICIAL DE JUSTICA|TABELIAO|CARTORIO',
        'Seguranca': r'POLICIAL|MILITAR|VIGILANTE|SEGURANCA|BOMBEIRO',
        'Cultura_Comunicacao': r'MUSICO|ATOR|ARTESAO|JORNALISTA|ESCULTOR|PUBLICITARIO|FOTOGRAFO|LOCUTOR'
    }
    df['grupo_ocupacao'] = 'Outros'
    if 'DS_OCUPACAO' in df.columns:
        for grupo, regex in ocup_map.items():
            df.loc[df['DS_OCUPACAO'].str.contains(regex, case=False, na=False), 'grupo_ocupacao'] = grupo

    # Guardar rótulos para gráficos
    df['grupo_ocupacao_cat'] = df['grupo_ocupacao'].astype('category')
    df['perfil_grupo_cat'] = df['perfil_grupo'].astype('category')

    # Dummies com nomes slug
    oc_dum = pd.get_dummies(df['grupo_ocupacao'], prefix='oc', dtype=int)
    oc_dum.columns = [slug(c) for c in oc_dum.columns]
    pf_dum = pd.get_dummies(df['perfil_grupo'], prefix='perfil', dtype=int)
    pf_dum.columns = [slug(c) for c in pf_dum.columns]
    df = pd.concat([df, oc_dum, pf_dum], axis=1)

    # Variáveis regionais (valores médios por região)
    dados_regionais = {
        'regiao_norte':        {'escolaridade': 9.2,  'renda': 2421.7, 'idh': 0.6847, 'pib': 33123},
        'regiao_nordeste':     {'escolaridade': 8.3,  'renda': 2078.0, 'idh': 0.6487, 'pib': 25401},
        'regiao_sudeste':      {'escolaridade': 10.0, 'renda': 3514.0, 'idh': 0.7537, 'pib': 63327},
        'regiao_sul':          {'escolaridade': 10.1, 'renda': 3423.7, 'idh': 0.7563, 'pib': 55942},
        'regiao_centro_oeste': {'escolaridade': 10.1, 'renda': 3604.0, 'idh': 0.7533, 'pib': 65651},
    }
    for var in ['escolaridade_regiao', 'renda_regional', 'idh_regional', 'pib_percapita_regional']:
        df[var] = np.nan
    for regiao, valores in dados_regionais.items():
        mask = df.get(regiao, 0) == 1
        df.loc[mask, 'escolaridade_regiao'] = valores['escolaridade']
        df.loc[mask, 'renda_regional'] = valores['renda']
        df.loc[mask, 'idh_regional'] = valores['idh']
        df.loc[mask, 'pib_percapita_regional'] = valores['pib']

    # Variáveis dependentes
    soma_complex_cols = [c for c in ['MULTIMERCADOS','RENDA_VARIAVEL','INVEST_ALTERNATIVOS','investimento_exterior'] if c in df.columns]
    soma_total_cols   = [c for c in ['RENDA_FIXA_POS_CDI','RENDA_FIXA_PRE','RENDA_FIXA_INFLACAO','MULTIMERCADOS','RENDA_VARIAVEL','INVEST_ALTERNATIVOS','investimento_exterior'] if c in df.columns]
    df['soma_complex'] = df[soma_complex_cols].sum(axis=1, skipna=True) if soma_complex_cols else 0.0
    df['soma_total']   = df[soma_total_cols].sum(axis=1, skipna=True) if soma_total_cols else 0.0
    df['diver'] = (df['soma_complex'] / df['soma_total']).replace([np.inf, -np.inf], np.nan).fillna(0.0).clip(upper=1.0)
    df['complex'] = (df['soma_complex'] > 0).astype(int)

    # Índice de painel
    if 'anomes' not in df.columns:
        raise KeyError("Coluna 'anomes' (YYYYMM) não encontrada.")
    df['anomes'] = pd.to_datetime(df['anomes'].astype(int).astype(str), format='%Y%m')
    df = df.set_index(['id_cliente', 'anomes']).sort_index()
    df = df.loc[~df.index.duplicated(keep='first')]

    # Delta de renda individual e skew proxy
    if 'renda' not in df.columns:
        raise KeyError("Coluna 'renda' (renda individual mensal ou similar) não encontrada.")
    df['log_renda_ind'] = np.log(df['renda'] + 1)
    df['delta_y'] = df.groupby(level='id_cliente')['log_renda_ind'].diff()
    df['ano'] = df.index.get_level_values('anomes').year
    df['delta_y_anual'] = df.groupby(['id_cliente', 'ano'])['delta_y'].transform('mean')
    df['media_dy'] = df.groupby(['regiao_codigo', 'ano'])['delta_y_anual'].transform('mean')
    df['sd_dy']    = df.groupby(['regiao_codigo', 'ano'])['delta_y_anual'].transform('std')
    df['delta_y_pad'] = (df['delta_y_anual'] - df['media_dy']) / df['sd_dy']
    df['skew_aux'] = df['delta_y_pad']**3
    df['grupo_n']  = df.groupby(['regiao_codigo', 'ano'])['skew_aux'].transform('count')
    df['skew']     = df.groupby(['regiao_codigo', 'ano'])['skew_aux'].transform('mean')
    df.loc[df['grupo_n'] < 30, 'skew'] = np.nan
    df['skew_final'] = df['skew'].where(df['ano'] > 2021)
    df['skew_media_regional'] = df.groupby('regiao_codigo')['skew_final'].transform('mean')
    df['skew_proxy'] = df['skew'].fillna(df['skew_media_regional'])

    # Logs + winsor
    df['ln_diver'] = np.log(df['diver'] + 0.01)
    df['ln_renda'] = np.log(df['renda_regional'].fillna(1))
    df['ln_ESC']   = np.log(df['escolaridade_regiao'].fillna(1))
    df['ln_IDH']   = np.log(df['idh_regional'].fillna(1))
    df['ln_PIB']   = np.log(df['pib_percapita_regional'].fillna(1))

    for var in ['ln_diver', 'ln_renda', 'ln_ESC', 'ln_IDH', 'ln_PIB']:
        df[f'{var}_w'] = winsorize_series(df[var])

    return df


# ------------------------------
# 3) Modelagem
# ------------------------------
def run_models(df: pd.DataFrame):
    prints = []

    # Matriz de correlação (salvar figura)
    corr_vars = [c for c in ['ln_diver_w','ln_renda_w','ln_ESC_w','ln_IDH_w','ln_PIB_w','idade_int','sexo_dummy','skew_proxy'] if c in df.columns]
    corr = df[corr_vars].corr()
    fig_path = os.path.join(RESULTS_DIR, "matriz_correlacao.png")
    try:
        plt.figure(figsize=(10, 8))
        if sns is not None:
            ax = sns.heatmap(corr, annot=True, fmt=".2f")
        else:
            # fallback simples com matplotlib
            plt.imshow(corr, interpolation='nearest')
            plt.colorbar()
            plt.xticks(range(len(corr_vars)), corr_vars, rotation=45, ha='right')
            plt.yticks(range(len(corr_vars)), corr_vars)
        plt.title('Matriz de Correlação')
        plt.tight_layout()
        plt.savefig(fig_path, dpi=180)
        plt.close()
        prints.append(f"[OK] Matriz de correlação salva em: {fig_path}")
    except Exception as e:
        prints.append(f"[WARN] Falha ao salvar matriz de correlação: {e}")

    # Fórmulas
    formula_base = 'ln_diver_w ~ ln_renda_w + ln_IDH_w + sexo_dummy + idade_int'

    # Pooled
    pooled = PooledOLS.from_formula(formula_base, data=df).fit(cov_type='robust')
    prints.append("\n[POOLED]\n" + str(pooled.summary))

    # FE
    fe = PanelOLS.from_formula(formula_base + ' + EntityEffects', data=df).fit(cov_type='clustered', cluster_entity=True)
    prints.append("\n[FE]\n" + str(fe.summary))

    # RE
    re = RandomEffects.from_formula(formula_base, data=df).fit(cov_type='clustered', cluster_entity=True)
    prints.append("\n[RE]\n" + str(re.summary))

    # Hausman
    try:
        stat, dof, p = hausman(fe, re)
        prints.append(f"\n[HAUSMAN] chi2({dof})={stat:.2f}, p={p:.4f}")
    except Exception as e:
        prints.append(f"\n[HAUSMAN] falhou: {e}")

    # H1
    formula_h1 = (
        'ln_diver_w ~ complex + ln_ESC_w + ln_renda_w + '
        'regiao_norte + regiao_nordeste + regiao_sul + regiao_centro_oeste + '
        'sexo_dummy + idade_int + EntityEffects'
    )
    h1 = PanelOLS.from_formula(formula_h1, data=df).fit(cov_type='clustered', cluster_entity=True)
    prints.append("\n[H1/H1a]\n" + str(h1.summary))

    # H2
    formula_h2 = (
        'ln_diver_w ~ skew_proxy + ln_ESC_w + ln_renda_w + '
        'regiao_norte + regiao_nordeste + regiao_sul + regiao_centro_oeste + '
        'sexo_dummy + idade_int + EntityEffects'
    )
    h2 = PanelOLS.from_formula(formula_h2, data=df).fit(cov_type='clustered', cluster_entity=True)
    prints.append("\n[H2]\n" + str(h2.summary))

    # H3 (alta renda individual)
    if 'renda' in df.columns:
        df_h3 = df[df['renda'] > 20000]
        formula_h3 = (
            'ln_diver_w ~ ln_IDH_w + ln_renda_w + '
            'regiao_norte + regiao_nordeste + regiao_sul + regiao_centro_oeste + '
            'sexo_dummy + idade_int + EntityEffects'
        )
        h3 = PanelOLS.from_formula(formula_h3, data=df_h3).fit(cov_type='clustered', cluster_entity=True)
        prints.append("\n[H3]\n" + str(h3.summary))

    # Estatísticas descritivas
    desc_cols = [c for c in [
        'diver','ln_diver_w','ln_renda_w','ln_ESC_w','ln_IDH_w','idade_int','sexo_dummy','complex','skew_proxy',
        'perfil_conservador','perfil_moderado','perfil_arrojado'
    ] if c in df.columns]
    desc = df[desc_cols].describe().T
    desc_path = os.path.join(RESULTS_DIR, "estatisticas_descritivas.csv")
    desc.to_csv(desc_path, encoding='utf-8')
    prints.append(f"\n[DESCRITIVAS] salvo em: {desc_path}")

    # Gráficos simples (salvar)
    try:
        if all(c in df.columns for c in ['ln_renda_w', 'ln_diver_w']):
            plt.figure(figsize=(6,4))
            plt.scatter(df['ln_renda_w'], df['ln_diver_w'], s=6, alpha=0.3)
            # linha de tendência via np.polyfit
            m, b = np.polyfit(df['ln_renda_w'].replace([np.inf,-np.inf], np.nan).dropna(),
                              df['ln_diver_w'].replace([np.inf,-np.inf], np.nan).dropna(), 1)
            xvals = np.linspace(df['ln_renda_w'].min(), df['ln_renda_w'].max(), 200)
            plt.plot(xvals, m*xvals + b)
            plt.title('Diversificação vs. Renda (winsor)')
            plt.xlabel('ln_renda_w')
            plt.ylabel('ln_diver_w')
            plt.tight_layout()
            fig2 = os.path.join(RESULTS_DIR, "scatter_diver_vs_renda.png")
            plt.savefig(fig2, dpi=160)
            plt.close()
            prints.append(f"[OK] Gráfico scatter salvo em: {fig2}")
    except Exception as e:
        prints.append(f"[WARN] Falha ao salvar gráfico scatter: {e}")

    # Boxplots por ocupação/perfil (seaborn se disponível)
    try:
        if sns is not None and 'grupo_ocupacao_cat' in df.columns:
            plt.figure(figsize=(10,5))
            sns.boxplot(x='grupo_ocupacao_cat', y='diver', data=df.reset_index())
            plt.xticks(rotation=45, ha='right')
            plt.title('Diversificação por Grupo de Ocupação')
            plt.tight_layout()
            fig3 = os.path.join(RESULTS_DIR, "boxplot_diver_ocupacao.png")
            plt.savefig(fig3, dpi=160); plt.close()
            prints.append(f"[OK] Boxplot ocupação salvo em: {fig3}")
        if sns is not None and 'perfil_grupo_cat' in df.columns:
            plt.figure(figsize=(8,5))
            sns.boxplot(x='perfil_grupo_cat', y='diver', data=df.reset_index())
            plt.title('Diversificação por Perfil do Investidor')
            plt.tight_layout()
            fig4 = os.path.join(RESULTS_DIR, "boxplot_diver_perfil.png")
            plt.savefig(fig4, dpi=160); plt.close()
            prints.append(f"[OK] Boxplot perfil salvo em: {fig4}")
    except Exception as e:
        prints.append(f"[WARN] Falha ao salvar boxplots: {e}")

    # Salvar sumários em arquivo texto
    sum_path = os.path.join(RESULTS_DIR, "sumarios_modelos.txt")
    with open(sum_path, "w", encoding="utf-8") as fh:
        fh.write("\n".join(prints))
    print(f"Sumários salvos em: {sum_path}")
    for p in prints:
        print(p)

    return dict(pooled=pooled, fe=fe, re=re)


# ------------------------------
# Main
# ------------------------------
def main():
    df = load_data(INPUT_PATH)
    df = prepare_engineer(df)

    # Checagens rápidas
    print("Total de clientes:", df.index.get_level_values('id_cliente').nunique())
    print("Período:", df.index.get_level_values('anomes').min().date(), "→", df.index.get_level_values('anomes').max().date())
    print("Soma dummies regionais:\n", df.filter(regex=r'^regiao_').sum())

    run_models(df)


if __name__ == "__main__":
    main()


[bootstrap] executando: /Users/macvini/Library/CloudStorage/OneDrive-Pessoal/Repos/disserta_dados/.venv/bin/python -m pip install --disable-pip-version-check --no-input -r requirements.txt
[bootstrap] Instalação concluída. Reinicie o kernel para carregar os novos pacotes.
Total de clientes: 99817
Período: 2021-01-01 → 2024-12-01
Soma dummies regionais:
 regiao_norte              98208
regiao_nordeste          652752
regiao_sudeste          2659824
regiao_sul               917148
regiao_centro_oeste      463008
regiao_codigo          15366816
dtype: int64


/var/folders/1t/dhbsy99x3cv29zhft5ks9w7c0000gn/T/ipykernel_46699/1627058761.py:71: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  stat = float(diff.T @ np.linalg.pinv(V) @ diff)


Sumários salvos em: ./resultados_python/sumarios_modelos.txt
[OK] Matriz de correlação salva em: ./resultados_python/matriz_correlacao.png

[POOLED]
                          PooledOLS Estimation Summary                          
Dep. Variable:             ln_diver_w   R-squared:                        0.7332
Estimator:                  PooledOLS   R-squared (Between):              0.7750
No. Observations:             4791216   R-squared (Within):           -2.871e-05
Date:                Sat, Aug 30 2025   R-squared (Overall):              0.7332
Time:                        09:47:00   Log-likelihood                 -9.39e+06
Cov. Estimator:                Robust                                           
                                        F-statistic:                   3.292e+06
Entities:                       99817   P-value                           0.0000
Avg Obs:                       48.000   Distribution:               F(4,4791212)
Min Obs:                       48.000    